In [ ]:
# Imports
import numpy as np
import pandas as pd
from tqdm import tqdm  
from skyfield.api import load, wgs84
from skyfield.api import EarthSatellite
from datetime import datetime
import configparser
import json
import requests
import time
import urllib

In [ ]:
## LINK TO DATA FOLDER:-https://drive.google.com/drive/folders/1HExaz_W_dHEDSngdqHxW9UAyMazoX6ab?usp=sharing

## Please add it to the same folder as-"Maneuver_Detections"

In [ ]:
df = pd.read_csv('data(1)/filtered_elset.csv')
df.drop([df.columns[0]], axis=1 , inplace=True)
df.head()

,OBJECT_NAME,NORAD_ID,YEAR LAUNCH,EPOCH,INCLINATION(i),RIGHT ASCENSION ASCENDING NODE(w),ECCENTRICITY(e),ARGUMENT PERIGEE(omega),FIRST DERIVATIVE MEAN MOTION(n_dot),SECOND DERIVATIVE MEAN MOTION(n_ddot),MEAN ANOMALY(M),MEAN MOTION(n),DRAG TERM(B*),REVOLUTION(rev),TLE_FIRST_LINES,TLE_SECOND_LINES,TLE_THIRD_LINES
0,VANGUARD 1,5,58002B,21042.550355,34.2457,211.6733,0.184707,348.3317,-8.900000e-07,0.0,7.9494,10.848569,-10.5760,13249,0 VANGUARD 1,1 5U 58002B 21042.55035550 -.00000089 0...,2 5 34.2457 211.6733 1847075 348.3317 7...
1,VANGUARD 2,11,59001A,21042.875208,32.8652,236.4804,0.146771,163.1695,2.500000e-06,0.0,202.3396,11.857480,12.3720,34028,0 VANGUARD 2,1 00011U 59001A 21042.87520771 .00000250 0...,2 00011 32.8652 236.4804 1467709 163.1695 202...
2,VANGUARD 3,20,59007A,21043.437696,33.3449,47.0561,0.166567,99.8465,4.900000e-07,0.0,279.3422,11.557979,2.6295,84140,0 VANGUARD 3,1 20U 59007A 21043.43769585 .00000049 0...,2 20 33.3449 47.0561 1665671 99.8465 279...
3,EXPLORER 7,22,59009A,21042.889513,50.2840,58.4215,0.013875,93.7045,-1.600000e-07,0.0,267.9779,14.954023,2.2213,61910,0 EXPLORER 7,1 00022U 59009A 21042.88951290 -.00000016 0...,2 00022 50.2840 58.4215 0138746 93.7045 267...
4,TIROS 1,29,60002B,21042.696355,48.3785,259.5211,0.002395,177.0037,-1.170000e-06,0.0,183.1022,14.744034,1.3635,38438,0 TIROS 1,1 00029U 60002B 21042.69635468 -.00000117 0...,2 00029 48.3785 259.5211 0023954 177.0037 183...


In [ ]:
dframe = pd.read_csv('data(1)/launch_sites.csv')
#dframe.drop([dframe.columns[0]], axis=1 , inplace=True)
dframe.head()
dframe_cp = dframe.copy()
dframe_cp.head()

,Unnamed: 0,LONGITUDE,LATITUDE
0,Hammaguir,-2.835810,31.09951
1,Reggane,0.276910,26.71895
2,Kapani Tonneo,28.527660,-7.92587
3,Jabal Hamzah,30.605139,30.12575
4,San Marco platform,40.213400,-2.94080


In [ ]:
# SPACE_TRACK API

class MyError(Exception):
    def __init___(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args


uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query"

# Use configparser package to pull in the ini file (pip install configparser)
config = configparser.ConfigParser()
config.read("data(1)/SLTrack.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
siteCred = {'identity': configUsr, 'password': configPwd}

def getTLESatellites(norad_id):

    with requests.Session() as session:
        # run the session in a with block to force session to close if we exit
        # need to log in first. note that we get a 200 to say the web site got the data, not that we are logged in
        resp = session.post(uriBase + requestLogin, data = siteCred)
        if resp.status_code != 200:
            raise MyError(resp, "POST fail on login")

        # this query picks up TLEs for all satellites from the catalog. Note - a 401 failure shows you have bad credentials
        # Enter the fields for which query must be built

        query_link = ("/class/tle/NORAD_CAT_ID/" + str(norad_id) + "/orderby/EPOCH asc/limit/10/format/3le/emptyresult/show")

        resp = session.get(uriBase + requestCmdAction + query_link)
        
        if resp.status_code != 200:
            time.sleep(5)
            return getTLESatellites(norad_id)

        # resp.text contains the required Tles
        data = resp.text
        session.close()
    return data

In [ ]:
epoch_list = []
tle_line1_list =[]
tle_line2_list = []
tle_line3_list = []
launch_sites= []
starting_ind = 5373
ending_ind = 5380
'''
for i in tqdm(range(starting_ind,ending_ind)):
    time.sleep(0.2)
    norad_id = df.iloc[i]['NORAD_ID']
    starting_tles = getTLESatellites(norad_id)

    lines = starting_tles.splitlines()

    tle_line1 = lines[0]
    tle_line2 = lines[1]
    tle_line3 = lines[2]

    tle_line2_ = tle_line2.split()
    epoch = float(tle_line2_[4])
    epoch_list.append(epoch)
    tle_line1_list.append(tle_line1)
    tle_line2_list.append(tle_line2)
    tle_line3_list.append(tle_line3)
'''

time.sleep(0.2)
norad_id = df.iloc[5373]['NORAD_ID']
starting_tles = getTLESatellites(norad_id)

lines = starting_tles.splitlines()

for i in range(0,len(lines)-2,3):
    tle_line1 = lines[i]
    tle_line2 = lines[i+1]
    tle_line3 = lines[i+2]

    tle_line1_list.append(tle_line1)
    tle_line2_list.append(tle_line2)
    tle_line3_list.append(tle_line3)



In [ ]:
def find_closest_subpoint(longitude, latitude):
    long_threshold = 1.0 # deg
    lat_threshold = 1.0 # deg
    closest_point = None
    min_dev = 1000.0
    closest_launch_sites= []
    for i in range(dframe.shape[0]):
        launch_site_long = dframe.iloc[i][1]
        launch_site_lat = dframe.iloc[i][2]
        if abs(launch_site_long - longitude)<=long_threshold and abs(launch_site_lat - latitude)<=lat_threshold:
            if abs(launch_site_long - longitude)  + abs(launch_site_lat - latitude) < min_dev:
                min_dev = abs(launch_site_long - longitude)  + abs(launch_site_lat - latitude)
                closest_point = dframe.iloc[i][0]
            closest_launch_sites.append(dframe.iloc[i][0])

    return closest_point, closest_launch_sites, min_dev

In [ ]:
launch_dates_list = ['2019-11-11','2019-11-11','2019-11-11','2019-11-11','2019-11-11','2019-11-11', '2019-11-11', '2019-11-11', '2019-11-11', '2019-11-11']

In [ ]:
from dateutil import rrule
from datetime import timedelta

for i in tqdm(range(1)):
    date = datetime.fromisoformat(launch_dates_list[i])
    startdate = date+timedelta(days=1)
    endate= date + timedelta(days=2)
    print('Launch Date: ', date)
    print('Launch Time: ', '15:00:00')

    tle_line1 = tle_line1_list[i]
    tle_line2 = tle_line2_list[i]
    tle_line3 = tle_line3_list[i]
    print('FIRST_TLE=', tle_line2)
    
    ts = load.timescale()
    satellite_skyfield = EarthSatellite(tle_line2, tle_line3, tle_line1, ts)
    launch_site1 = None
    min_dev = 1000.0
    min_long = 0.0
    min_lat = 0.0
    close_launch_sites = []

    for dt in rrule.rrule(rrule.MINUTELY, dtstart=startdate, until=endate):
        #print(dt)
        t = ts.utc(dt.year, dt.month, dt.day, dt.hour,dt.minute,dt.second)
        geocentric = satellite_skyfield.at(t)
        subpoint = wgs84.subpoint(geocentric)

 
        start_long = subpoint.longitude.degrees
        start_lat = subpoint.latitude.degrees

        launch_site, closest_launch_sites, min_dev_ = find_closest_subpoint(start_long, start_lat)
        close_launch_sites.extend(closest_launch_sites)
        close_launch_sites = list(set(close_launch_sites))
    
        if min_dev_ < min_dev:
            launch_site1 = launch_site
            min_dev = min_dev_
            min_long = start_long
            min_lat = start_lat
    

In [ ]:
dframe_cp = dframe_cp.reindex(dframe_cp.columns.tolist() + ['min err tle1','min err tle2', 'min err tle3','min err tle4','min err tle5','min err tle6','min err tle7','min err tle8','min err tle9','min err tle10'], axis=1)  

In [ ]:
from dateutil import rrule
from datetime import timedelta

print(dframe_cp.shape)
for i in tqdm(range(dframe_cp.shape[0])):
    err_list = []
    launch_site_long = dframe_cp.iloc[i][1]
    launch_site_lat = dframe_cp.iloc[i][2]
    for j in range(10):
        date = datetime.fromisoformat(launch_dates_list[j])
        startdate = date+timedelta(days=1)
        endate= startdate + timedelta(hours=4)

        tle_line1 = tle_line1_list[j]
        tle_line2 = tle_line2_list[j]
        tle_line3 = tle_line3_list[j]
    
        ts = load.timescale()
        satellite_skyfield = EarthSatellite(tle_line2, tle_line3, tle_line1, ts)
        min_err = 10000.0
        min_long = 0.0
        min_lat = 0.0

        dt = startdate
        while dt<=endate:
        #for dt in rrule.rrule(rrule.SECONDLY, dtstart=startdate, until=endate):
            t = ts.utc(dt.year, dt.month, dt.day, dt.hour,dt.minute,dt.second)
            geocentric = satellite_skyfield.at(t)
            subpoint = wgs84.subpoint(geocentric)
 
            start_long = subpoint.longitude.degrees
            start_lat = subpoint.latitude.degrees

            error = abs(launch_site_long - start_long)  + abs(launch_site_lat - start_lat)
   
            if error < min_err:
                min_err = error
            dt = dt + timedelta(seconds=30)

        #print(dframe_cp.iloc[i][j+3]) 
        dframe_cp.iat[i, j+3] = min_err    
dframe_cp.head()

(175, 13)


100%|██████████| 175/175 [14:06<00:00,  4.84s/it]


,Unnamed: 0,LONGITUDE,LATITUDE,min err tle1,min err tle2,min err tle3,min err tle4,min err tle5,min err tle6,min err tle7,min err tle8,min err tle9,min err tle10
0,Hammaguir,-2.835810,31.09951,81.195096,81.674159,81.619885,81.329774,81.663166,81.767656,82.681142,81.350511,81.493580,83.422523
1,Reggane,0.276910,26.71895,75.518444,76.005314,75.947772,75.981326,76.612695,76.583832,77.204067,78.050829,77.452953,78.723177
2,Kapani Tonneo,28.527660,-7.92587,38.836464,32.678813,35.722519,25.779329,23.804137,24.069201,24.489917,27.735483,26.574716,31.691692
3,Jabal Hamzah,30.605139,30.12575,69.762087,70.371732,70.247688,61.753470,57.738107,56.952507,55.300712,54.055326,52.977281,57.978399
4,San Marco platform,40.213400,-2.94080,34.931200,35.532113,35.438943,24.173335,15.585445,15.852313,16.267489,19.519772,18.352527,23.480227


In [ ]:
dframe_cp.to_csv('data(1)/dframe_cp.csv')

In [ ]:
dframe_cp = dframe_cp.reindex(dframe_cp.columns.tolist() + ['MEAN', 'STD'], axis=1)  

In [ ]:
import statistics
for i in range(dframe_cp.shape[0]):
    err_list = []
    for j in range(3,13):
        err_list.append(dframe_cp.iloc[i][j])
    mean = sum(err_list) / len(err_list)
    std = sum([((x - mean) ** 2) for x in err_list]) / len(err_list)
    dframe_cp.iat[i,13] = mean
    dframe_cp.iat[i,14] = std

dframe_cp.head()
dframe_cp.to_csv('data(1)/dframe_cp.csv')

In [ ]:
dframe_cp.head()

,Unnamed: 0,LONGITUDE,LATITUDE,min err tle1,min err tle2,min err tle3,min err tle4,min err tle5,min err tle6,min err tle7,min err tle8,min err tle9,min err tle10,MEAN,STD,MEAN,STD
0,Hammaguir,-2.835810,31.09951,81.195096,81.674159,81.619885,81.329774,81.663166,81.767656,82.681142,81.350511,81.493580,83.422523,81.819749,0.435609,81.819749,0.435609
1,Reggane,0.276910,26.71895,75.518444,76.005314,75.947772,75.981326,76.612695,76.583832,77.204067,78.050829,77.452953,78.723177,76.808041,0.960440,76.808041,0.960440
2,Kapani Tonneo,28.527660,-7.92587,38.836464,32.678813,35.722519,25.779329,23.804137,24.069201,24.489917,27.735483,26.574716,31.691692,29.138227,25.204043,29.138227,25.204043
3,Jabal Hamzah,30.605139,30.12575,69.762087,70.371732,70.247688,61.753470,57.738107,56.952507,55.300712,54.055326,52.977281,57.978399,60.713731,43.109811,60.713731,43.109811
4,San Marco platform,40.213400,-2.94080,34.931200,35.532113,35.438943,24.173335,15.585445,15.852313,16.267489,19.519772,18.352527,23.480227,23.913337,63.250294,23.913337,63.250294


In [ ]:
from dateutil import rrule
from datetime import timedelta

for i in tqdm(range(10)):
    date = datetime.fromisoformat(launch_dates_list[i])
    startdate = date+timedelta(days=1)
    endate= date + timedelta(days=2)
    print('Launch Date: ', date)
    print('Launch Time: ', '15:00:00')

    tle_line1 = tle_line1_list[i]
    tle_line2 = tle_line2_list[i]
    tle_line3 = tle_line3_list[i]
    print('FIRST_TLE=', tle_line2)
    
    ts = load.timescale()
    satellite_skyfield = EarthSatellite(tle_line2, tle_line3, tle_line1, ts)
    launch_site1 = None
    min_dev = 1000.0
    min_long = 0.0
    min_lat = 0.0
    close_launch_sites = []

    for dt in rrule.rrule(rrule.MINUTELY, dtstart=startdate, until=endate):
        #print(dt)
        t = ts.utc(dt.year, dt.month, dt.day, dt.hour,dt.minute,dt.second)
        geocentric = satellite_skyfield.at(t)
        subpoint = wgs84.subpoint(geocentric)

 
        start_long = subpoint.longitude.degrees
        start_lat = subpoint.latitude.degrees

        launch_site, closest_launch_sites, min_dev_ = find_closest_subpoint(start_long, start_lat)
        close_launch_sites.extend(closest_launch_sites)
        close_launch_sites = list(set(close_launch_sites))
    
        if min_dev_ < min_dev:
            launch_site1 = launch_site
            min_dev = min_dev_
            min_long = start_long
            min_lat = start_lat
    
 

    print('LAT FROM EARLIEST TLE=', min_lat, 'LONG FROM EARLIEST TLE=', min_long, 'SATNAME=', tle_line1[1:], 'LAUNCH-SITE= ', launch_site1, 'MIN_DEV=', min_dev)
    print('LAUNCH SITES THAT ARE CLOSE= ', close_launch_sites)
    print(len(close_launch_sites))
    #print('LAT FROM LATEST TLE=', latest_lat, 'LONG FROM LATEST TLE=', latest_long,'SATNAME=', tle_line1[1:], 'LAUNCH-SITE= ', launch_site2)
    print("------------------------")


  0%|          | 0/10 [00:00<?, ?it/s]

Launch Date:  2019-11-11 00:00:00
Launch Time:  15:00:00
FIRST_TLE= 1 44713U 19074A   19318.91668981  .00114330  15931-4  36593-3 0  9994


 10%|█         | 1/10 [01:49<16:24, 109.33s/it]

LAT FROM EARLIEST TLE= 8.408175417099418 LONG FROM EARLIEST TLE= 170.88525789900802 SATNAME=  STARLINK-1007 LAUNCH-SITE=  Bigen island MIN_DEV= 0.2011775180913773
LAUNCH SITES THAT ARE CLOSE=  ['Red Lake', 'Coronie', 'Point Arguello', '#82', 'San Clemente', 'The Needles', 'Jabal Hamzah', 'Taiyuan', 'Bigen island', 'Vandenberg', 'Blizna', 'Woomera', 'Overberg', 'Hespenbusch', 'Palmachim', 'Raketenflugplatz Tegel', 'Guam']
17
------------------------
Launch Date:  2019-11-11 00:00:00
Launch Time:  15:00:00
FIRST_TLE= 1 44713U 19074A   19319.25001157  .00149286  00000-0  47594-3 0  9996


 10%|█         | 1/10 [02:23<21:34, 143.82s/it]


KeyboardInterrupt: ignored

In [ ]:
print(tle_line1_list)

In [ ]:
from skyfield import api
from skyfield.positionlib import ICRF, Geocentric
from skyfield.constants import (AU_M, ERAD, DEG2RAD,
                                IERS_2010_INVERSE_EARTH_FLATTENING, tau)
from skyfield.units import Angle

from numpy import einsum, sqrt, arctan2, pi, cos, sin

def reverse_terra(xyz_au, gast, iterations=3):
    """Convert a geocentric (x,y,z) at time `t` to latitude and longitude.
    Returns a tuple of latitude, longitude, and elevation whose units
    are radians and meters.  Based on Dr. T.S. Kelso's quite helpful
    article "Orbital Coordinate Systems, Part III":
    https://www.celestrak.com/columns/v02n03/
    """
    x, y, z = xyz_au
    R = sqrt(x*x + y*y)

    lon = (arctan2(y, x) - 15 * DEG2RAD * gast - pi) % tau - pi
    lat = arctan2(z, R)

    a = ERAD / AU_M
    f = 1.0 / IERS_2010_INVERSE_EARTH_FLATTENING
    e2 = 2.0*f - f*f
    i = 0
    C = 1.0
    while i < iterations:
        i += 1
        C = 1.0 / sqrt(1.0 - e2 * (sin(lat) ** 2.0))
        lat = arctan2(z + a * C * e2 * sin(lat), R)
    elevation_m = ((R / cos(lat)) - a * C) * AU_M
    earth_R = (a*C)*AU_M
    return lat, lon, elevation_m, earth_R

def subpoint(self, iterations):
    """Return the latitude an longitude directly beneath this position.

    Returns a :class:`~skyfield.toposlib.Topos` whose ``longitude``
    and ``latitude`` are those of the point on the Earth's surface
    directly beneath this position (according to the center of the
    earth), and whose ``elevation`` is the height of this position
    above the Earth's center.
    """
    if self.center != 399:  # TODO: should an __init__() check this?
        raise ValueError("you can only ask for the geographic subpoint"
                            " of a position measured from Earth's center")
    t = self.t
    xyz_au = einsum('ij...,j...->i...', t.M, self.position.au)
    lat, lon, elevation_m, self.earth_R = reverse_terra(xyz_au, t.gast, iterations)

    from skyfield.toposlib import Topos
    return Topos(latitude=Angle(radians=lat),
                    longitude=Angle(radians=lon),
                    elevation_m=elevation_m)

def earth_radius(self):
    return self.earth_R

def satellite_visiable_area(earth_radius, satellite_elevation):
    """Returns the visible area from a satellite in square meters.

    Formula is in the form is 2piR^2h/R+h where:
        R = earth radius
        h = satellite elevation from center of earth
    """
    return ((2 * pi * ( earth_radius ** 2 ) * 
            ( earth_radius + satellite_elevation)) /
            (earth_radius + earth_radius + satellite_elevation))

'''
stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
satellites = api.load.tle(stations_url)
satellite = satellites['ISS (ZARYA)']
print(satellite)

ts = api.load.timescale()
t = ts.now()

geocentric = satellite.at(t)
geocentric.subpoint = subpoint.__get__(geocentric, Geocentric)
geocentric.earth_radius = earth_radius.__get__(geocentric, Geocentric)

geodetic_sub = geocentric.subpoint(3)

print('Geodetic latitude:', geodetic_sub.latitude)
print('Geodetic longitude:', geodetic_sub.longitude)
print('Geodetic elevation (m)', int(geodetic_sub.elevation.m))
print('Geodetic earth radius (m)', int(geocentric.earth_radius()))

geocentric_sub = geocentric.subpoint(0)
print('Geocentric latitude:', geocentric_sub.latitude)
print('Geocentric longitude:', geocentric_sub.longitude)
print('Geocentric elevation (m)', int(geocentric_sub.elevation.m))
print('Geocentric earth radius (m)', int(geocentric.earth_radius()))
print('Visible area (m^2)', satellite_visiable_area(geocentric.earth_radius(), 
                                                    geocentric_sub.elevation.m))
'''

In [ ]:
for i in range(ending_ind-starting_ind):
    date = datetime.fromisoformat(launch_dates_list[i])
    print('Launch Date: ', date.year, date.month, date.day)
    print('Launch Time: ', '15:00:00')

    tle_line1 = tle_line1_list[i]
    tle_line2 = tle_line2_list[i]
    tle_line3 = tle_line3_list[i]
    print('FIRST_TLE=', tle_line2)
    
    ts = load.timescale()
    t = ts.utc(date.year, date.month, date.day, 15,00,00)
    satellite = EarthSatellite(tle_line2, tle_line3, tle_line1, ts)
    print(satellite)
    geocentric = satellite.at(t)

    geocentric.subpoint = subpoint.__get__(geocentric, Geocentric)
    geocentric.earth_radius = earth_radius.__get__(geocentric, Geocentric)

    geodetic_sub = geocentric.subpoint(3)

    print('Geodetic latitude:', geodetic_sub.latitude)
    print('Geodetic longitude:', geodetic_sub.longitude)
    print('Geodetic elevation (m)', int(geodetic_sub.elevation.m))
    print('Geodetic earth radius (m)', int(geocentric.earth_radius()))

    geocentric_sub = geocentric.subpoint(0)
    print('Geocentric latitude:', geocentric_sub.latitude)
    print('Geocentric longitude:', geocentric_sub.longitude)
    print('Geocentric elevation (m)', int(geocentric_sub.elevation.m))
    print('Geocentric earth radius (m)', int(geocentric.earth_radius()))
    print('Visible area (m^2)', satellite_visiable_area(geocentric.earth_radius(), 
                                                    geocentric_sub.elevation.m))
    print("-------------------")